In [4]:
import numpy as np
import pandas as pd
import graph_tool.all as gt
import json
import matplotlib

In [9]:
with open('./../data/java.json') as f:
  data = json.load(f)

link_data = pd.DataFrame(data["links"])
link_data

,from,to,strength
0,org.apache.log4j,org.apache.log4j.spi.Filter,7
1,org.apache.log4j,org.apache.log4j.spi.LoggingEvent,11
2,org.apache.log4j,org.apache.log4j.spi.OptionHandler,3
3,org.apache.log4j,java.util.Collections,4
4,org.apache.log4j,java.util.Enumeration,5
...,...,...,...
7483,org.apache.logging.log4j.web,org.springframework.mock.web.MockServletContext,2
7484,org.apache.logging.log4j.web,javax.servlet.AsyncContext,1
7485,org.apache.logging.log4j.web,javax.servlet.http.HttpServlet,1
7486,org.apache.logging.log4j.web,javax.servlet.http.HttpServletResponse,1


In [16]:
g = gt.Graph()

v_list = {}
v_prop_list = []
v_prop = g.new_vertex_property("string")
for i, row in link_data.iterrows():
  if row['from'] not in v_list:
    v_list[row['from']] = g.add_vertex()
    v_prop[v_list[row['from']]] = row['from']
    v_prop_list.append(row['from'])
  if row['to'] not in v_list:
    v_list[row['to']] = g.add_vertex()
    v_prop[v_list[row['to']]] = row['to']
    v_prop_list.append(row['to'])

for i, row in link_data.iterrows():
  g.add_edge(v_list[row['from']], v_list[row['to']])

p = gt.sfdp_layout(g)
deg = g.degree_property_map("total")

In [13]:
import graph_tool.centrality as gtcentrality

In [19]:
page_rank_result = gtcentrality.pagerank(g)
p = gt.sfdp_layout(g, r=100)
gt.graph_draw(g, vertex_fill_color=page_rank_result, vertex_text=v_prop,vertex_text_position = 0,
              pos=p,
              vertex_size=gt.prop_to_size(page_rank_result, mi=5, ma=15),
              vorder=page_rank_result, vcmap=matplotlib.cm.gist_heat,
              output="page_rank_result.pdf")

<VertexPropertyMap object with value type 'vector<double>', for Graph 0x1c02c1060, at 0x1c02c2c20>

In [15]:
def show_page_rank_result():
  page_rank_result_array = page_rank_result.get_array()
  dic = {}
  for i in range(len(v_prop_list)):
    dic[v_prop_list[i]] = page_rank_result_array[i]

  sorted_dic = sorted(dic.items(), key=lambda x: x[1], reverse=True)
  for i in range(len(v_prop_list)):
    # print(sorted_dic[i])
    print("{0}位: {1} page_rank: {2}".format(i+1, sorted_dic[i][0], sorted_dic[i][1]))

show_page_rank_result()

1位: org.apache.logging.log4j.Logger page_rank: 0.0022330856936899216
2位: java.util.List page_rank: 0.0013537669264779228
3位: java.util.Map page_rank: 0.0012667672658185833
4位: org.apache.logging.log4j.status.StatusLogger page_rank: 0.0011991363906169107
5位: org.osgi.framework.launch.FrameworkFactory page_rank: 0.0010897528468011423
6位: org.apache.logging.log4j.core.config.plugins.Plugin page_rank: 0.0010891838069916966
7位: org.apache.logging.log4j.Level page_rank: 0.0010371638519600065
8位: org.apache.logging.log4j.util.Strings page_rank: 0.000994171381208846
9位: org.apache.logging.log4j.core.LogEvent page_rank: 0.0009084708679364477
10位: java.util.Arrays page_rank: 0.0009059629995364616
11位: org.junit.Test page_rank: 0.0008718660500559844
12位: org.apache.logging.log4j.LogManager page_rank: 0.0008617161654293976
13位: java.io.IOException page_rank: 0.0008464231658183199
14位: org.apache.log4j.rewrite.RewritePolicy page_rank: 0.000834917294266939
15位: org.apache.logging.log4j.core.time.Pre